# CMAP EMAT Running Experiments

In [ ]:
import emat
import os
import pandas as pd
import numpy as np
import gzip
from emat.util.show_dir import show_dir, show_file_contents

In [ ]:
import logging
from emat.util.loggers import log_to_stderr
log = log_to_stderr(logging.INFO)

## Connecting to the Model

The interface for this model is located in the `cmap_emat.py`
module, which we will import into this notebook.  This file is extensively
documented in comments, and is a great starting point for new users
who want to edit this interface.

In [ ]:
import cmap_emat

Within this module, you will find a definition for the 
`CMAP_EMAT_Model` class.

We initialize an instance of the model interface object.
If you look at the module code, you'll note the `__init__` function
does a number of things, including
loading the scope, and creating a SQLite database to work within.

In [ ]:
fx = cmap_emat.CMAP_EMAT_Model()

In [ ]:
fx.db.log("running-experiments notebook init")

## Multiprocessing for Running Multiple Experiments



### Automatic Multiprocessing for Running Multiple Experiments

The examples above are all essentially single-process demonstrations of using TMIP-EMAT to run core model
experiments, either by running all in one single process, or by having a user manually instantiate a number 
of single processes.  If your core model itself is multi-threaded or otherwise is designed to make 
full use of your multi-core CPU, or if a single core model run will otherwise max out some
computational resource (e.g. RAM, disk space) then single process operation should be sufficient.

If, on the other hand, your model is such that you can run multiple independent instances of
the model side-by-side on the same machine, but you don't want to manage the process of manually, 
then you could benefit from a multiprocessing approach that uses the `dask.distributed` library.  To
demonstrate this, we'll create yet another small design of experiments to run.

In [ ]:
pending = fx.read_experiment_parameters(design_name='lhs', only_pending=True)
pending

The demo module is set up to facilitate distributed multiprocessing. During the `setup`
step, a copy
of the entire files-based model is made, and the model
is run there instead of in the master 'clean' directory.  This allows each worker to edit the files
independently and simultaneously, without disturbing other parallel workers.



We are ready to run this model in parallel subprocesses.
To do so, we can use the `async_experiments` function, which will create a client (essentially, a master process that 
will request model runs of worker processes, and handle the results as they become available).  The 
`async_experiments` function accepts a `n_workers` argument, which sets the number of worker processes.  This should
be set at the maximum number of parallel instances of the core model that can be reasonably handled on this
computer.  Setting this maximum excessively high will potentially cause out-of-memory errors or slow down
the processes immensely, while setting it too low will leave computational resources unused.

If the overall set of experiments is being conducted on several different
computers, you may want to only conduct a subset of experiments on this
computer.

    fx.run_experiments(design=design.iloc[:20], evaluator=client)      # Run the first 20 experiments
    fx.run_experiments(design=design.iloc[20:40], evaluator=client)    # Run the second 20 experiments
    fx.run_experiments(design=design.iloc[40:60], evaluator=client)    # Run the third 20 experiments
    fx.run_experiments(design=design.iloc[60:], evaluator=client)      # Run the remaining experiments
    
You can also select a subset of experiments from the end of the list by indexing with negative numbers.

    fx.run_experiments(design=design.iloc[-4:], evaluator=client)      # Run the last 4 experiments
    fx.run_experiments(design=design.iloc[-8:-4], evaluator=client)    # Run the penultimate 4 experiments


In [ ]:
background = fx.async_experiments(
    design=pending,
    max_n_workers=15,
    stagger_start=90,
)

If something goes wrong, it may be desirable to restart the client without
killing this jupyter notebook itself.  The script that runs the core model
unfortunately doesn't respond well to SIGINT's, and may try to keep going 
for quite a while.  In this case, it might be necessary to manually kill all
the model runs and start over.

In [ ]:
# background.client.restart() # In case of emergency, un-comment